In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import re
import tensorflow as tf

In [ ]:
!nvidia-smi

Tue Mar 16 18:11:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    34W / 250W |  15393MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
train=pd.read_excel('Data_Train.xlsx')
test=pd.read_excel('Data_Test.xlsx')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7628 entries, 0 to 7627
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   STORY    7628 non-null   object
 1   SECTION  7628 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 119.3+ KB


In [ ]:
test.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   STORY   2748 non-null   object
dtypes: object(1)
memory usage: 21.6+ KB


In [ ]:
def cleaning(Inpdata):
    cleanedArticle1=re.sub(r'\\[?|$|(),"".@#=><|!]Â&*/-',r' ',Inpdata)
    cleanedArticle2=re.sub(r'https?://\S+|www\.\S+',r' ',cleanedArticle1)
    cleanedArticle3=re.sub(r'\b\w{1,2}\b', ' ',cleanedArticle2)
    cleanedArticle4=re.sub(r' +', ' ',cleanedArticle3)
    cleanedArticle5=re.sub(r'[^a-z A-Z]',r' ',cleanedArticle4)
    cleanedArticle6=cleanedArticle5.replace("  ", "")
    cleanedArticle7=cleanedArticle6.lower()
    return(cleanedArticle7)

In [ ]:
train['text_clean']=train['STORY'].apply(cleaning)
test['text_clean']=test['STORY'].apply(cleaning)

In [ ]:
train.head()

,STORY,SECTION,text_clean
0,But the most painful was the huge reversal in ...,3,but the most painful was the huge reversal fee...
1,How formidable is the opposition alliance amon...,0,how formidable the opposition alliance among c...
2,Most Asian currencies were trading lower today...,3,most asian currencies were trading lower today...
3,"If you want to answer any question, click on ‘...",1,you want answer any questionclickanswer after...
4,"In global markets, gold prices edged up today ...",3,global marketsgold prices edged today disappo...


In [ ]:
train=train.drop(labels=['STORY'], axis=1)
test=test.drop(labels=['STORY'], axis=1)

In [ ]:
train['SECTION'].nunique()

4

In [ ]:
from sklearn.model_selection import train_test_split

traindf,evaldf=train_test_split(train,test_size=0.25,random_state=5478)

In [ ]:
pip install pytorch-transformers

In [ ]:
pip install transformers

In [ ]:
from transformers import RobertaTokenizer,TFRobertaForSequenceClassification
from tqdm.gui import tqdm_gui

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

clean_text=[]

clean_label=[]

clean_text= [j for j in traindf['text_clean']]

clean_label=[w for w in traindf['SECTION']]

In [ ]:
inputs = tokenizer(clean_text, return_tensors="tf",padding=True,truncation=True,max_length=20)

In [ ]:
inputs['labels'] = tf.reshape(tf.constant(clean_label), (-1, 1))

In [ ]:
inputs.items

<bound method BatchEncoding.items of {'input_ids': <tf.Tensor: shape=(5721, 20), dtype=int32, numpy=
array([[    0, 19726,  1166, ...,    14,    40,     2],
       [    0,   119, 33523, ...,  3432,  1498,     2],
       [    0,   548,  1334, ..., 30826,  1543,     2],
       ...,
       [    0,   617,    26, ...,  1055,    71,     2],
       [    0,   627,  2523, ...,   537,   741,     2],
       [    0,   627,  1905, ...,  3340,  6212,     2]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(5721, 20), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>, 'labels': <tf.Tensor: shape=(5721, 1), dtype=int32, numpy=
array([[2],
       [1],
       [2],
       ...,
       [0],
       [0],
       [1]], dtype=int32)>}>

In [ ]:
traindf.shape, evaldf.shape

((5721, 2), (1907, 2))

In [ ]:
test_text=[]

test_text =[j for j in evaldf['text_clean']]

Xtest = tokenizer(test_text, return_tensors="tf",padding=True,truncation=True,max_length=20)

In [ ]:
model.compile()
outputs=model(inputs)

In [ ]:
loss = outputs.loss
logits = outputs.logits

In [ ]:
pred=model(Xtest)

In [ ]:
pred =np.argmax(pred.logits, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy Score",accuracy_score(pred,evaldf['SECTION'].values))

Accuracy Score 0.22024121657052964
